<a href="https://colab.research.google.com/github/damilojohn/Playlist_Generator/blob/main/ML/Generating_Afrobeats_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/damilojohn/Playlist_Generator

Cloning into 'Playlist_Generator'...
remote: Enumerating objects: 314, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 314 (delta 80), reused 108 (delta 47), pack-reused 152
Receiving objects: 100% (314/314), 925.61 KiB | 9.85 MiB/s, done.
Resolving deltas: 100% (130/130), done.


In [2]:
%cd /content/Playlist_Generator

/content/Playlist_Generator


In [3]:
!git config --global user.email "oduguwadamilolajohn40@gmail.com"
!git config --global user.name "damilojohn"

# Embeddings using HuggingFace Sentence Transformers

In [ ]:
#installing necessary libraries
!pip install sentence-transformers
!pip install wandb

In [ ]:
#songs csv
!wget https://raw.githubusercontent.com/damilojohn/Playlist_Generator/main/data/spotify_transformed.csv

--2023-06-21 02:35:16--  https://raw.githubusercontent.com/damilojohn/Playlist_Generator/main/data/spotify_transformed.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167653 (164K) [text/plain]
Saving to: ‘spotify_transformed.csv’

spotify_transformed 100%[===================>] 163.72K  --.-KB/s    in 0.02s   

2023-06-21 02:35:16 (6.91 MB/s) - ‘spotify_transformed.csv’ saved [167653/167653]



In [ ]:
%cd /content

/content


In [ ]:
#setting up wandb
!wandb login
import wandb

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
config = {
    'Name' : 'Embeddings for afrobeats lyrics',
    'Dataset' : 'spotify api lyrics',
    'Model' : 'msmarco-miniLM-pretrained',
    'notes':'No text cleaning was performed yet.creating a baseline performance'
}
wandb.init(project='Playlist Generator',entity='damilojohn',config=config)

In [ ]:
from sentence_transformers import SentenceTransformer,util
from tqdm.auto import tqdm
import pandas as pd
import transformers
import pickle
from huggingface_hub import hf_hub_download
import re

In [ ]:
#loading song lyrics into a dataset object
data_path = '/content/spotify_transformed.csv'

In [ ]:
embedder = SentenceTransformer('msmarco-distilbert-base-v4')

In [ ]:
songs= pd.read_csv(data_path)

In [ ]:
sample=songs

In [ ]:
def clean_verses(lyrics_uncleaned):
  """ takes a string of lyrics , split by newlines and removes all verses starting with '['"""
  lyrics_cleaned = lyrics_uncleaned.split('\n')
  lyrics_cleaned = [i for i in lyrics_cleaned if i]
  for i,lyrics in enumerate(lyrics_cleaned) :
    if not lyrics[0].isalpha():
      del lyrics_cleaned[i]
  return lyrics_cleaned


In [ ]:
songs['for_lyrics'][0]

In [ ]:
songs['for_lyrics'] = songs['for_lyrics'].apply(clean_verses)

In [ ]:
songs['for_lyrics'][0]

In [ ]:
songs['for_lyrics'].str.len()

In [ ]:
songs = songs[(songs['for_lyrics'].str.len() != 0 )]

In [ ]:
len(sample['for_lyrics'])

52

In [ ]:
def verses_and_indexes():
  """returns a dictionary of verses and indexes"""
  verses = []
  for i in songs['for_lyrics'].values:
    verses.append(i)
    verses = [i for i in verses if i]
  lyrics_for_embeddings = []
  for i in verses :
    for j in i :
      lyrics_for_embeddings.append(j)
  lyrics_and_indexes = {i:k for i,k in enumerate(lyrics_for_embeddings)}
  temp = dict(zip(songs.song_name,songs.for_lyrics))
  lyrics_and_titles={}
  for i,j in temp.items():
      for k in j:
          lyrics_and_titles[k]=i

  return lyrics_and_indexes,lyrics_for_embeddings,lyrics_and_titles

In [ ]:
def create_verses_df(verses):
  verse_df = pd.DataFrame.from_dict(verses,orient='index')
  verse_df.reset_index(level=0,inplace=True)
  verse_df.to_csv('/content/verses.csv')
  return verse_df


In [ ]:
new_verses,lyrics_for_embeddings,lyrics_and_titles = verses_and_indexes()

In [ ]:
create_verses_df(new_verses)

In [ ]:
embeddings = embedder.encode(lyrics_for_embeddings)
assert len(lyrics_for_embeddings) == len(embeddings) , 'embeddings and lyrics are mismatched '

In [ ]:
#trying out semantic search
query = ['relax and have fun']
query_embedding = embedder.encode(query)
hits = util.semantic_search(query_embedding,embeddings,top_k = 6)
hits

[[{'corpus_id': 4144, 'score': 0.660988986492157},
  {'corpus_id': 4140, 'score': 0.5234023928642273},
  {'corpus_id': 1997, 'score': 0.4780132472515106},
  {'corpus_id': 2007, 'score': 0.4780132472515106},
  {'corpus_id': 2015, 'score': 0.4780132472515106},
  {'corpus_id': 1974, 'score': 0.4780132472515106}]]

In [ ]:
for hit in hits[0]:
  print(lyrics_and_titles[new_verses[hit['corpus_id']]])

Jericho
Jericho
Likkle Riddim
Likkle Riddim
Likkle Riddim
Likkle Riddim


In [ ]:
with open(path='/content/verses.pkl','wb') as pk:
  pickle.dump(new_verses,pk)

In [ ]:
mappings = pd.DataFrame.from_dict(lyrics_and_titles,orient='index')

In [ ]:
mappings.to_csv('/content/song_mappings.csv')

In [ ]:
import pickle
path = '/content/em.pkl'
with open(path,'wb') as pk:
  pickle.dump(embeddings,pk)

In [ ]:
from huggingface_hub import login
login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from huggingface_hub import hf_hub_download

# Embeddings using Openai ADA-002 and Pinecone Vector DB


In [ ]:
!pip install -qU  pinecone-client datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.0 MB/s eta 0:00:00


In [ ]:
#building a pinecone index
import pinecone

#initialize connection to pinecone
pinecone.init(
    api_key='2b6ebef8-017d-450f-a640-6ed98916182f',
)
index_name = 'playlist-generator'
if index_name not in pinecone.list_indexes():
  pinecone.create_index(
      index_name,
      dimension = embedder.get_sentence_embedding_dimension(),
      metric = 'cosine')

In [ ]:
vectorDB = pinecone.Index(index_name)

In [ ]:
embeddings = embeddings.tolist()

In [ ]:
for index,verse in enumerate(lyrics_for_embeddings):
  embed = embeddings[index]
  song_name = mappings.loc[verse]
  to_upsert = list(zip(embed,song_name))
  _ = vectorDB.upsert(vectors=to_upsert)

TypeError: ignored

In [ ]:
#data
verses = hf_hub_download('damilojohn/Personal_Playlist_Generator',repo_type='dataset',filename = 'verses.csv'))